In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

columnsHistoricalbars = ['close','high','low','trade','open','time','volume','volumeWeighted']

stock_data = pd.read_json('../data/APPL.csv', convert_dates=['Time'])
stock_data = pd.json_normalize(stock_data['bars']['AAPL'])

stock_data.columns = columnsHistoricalbars
stock_data.head()

,close,high,low,trade,open,time,volume,volumeWeighted
0,105.35,105.368,102.00,351452,102.61,2016-01-04T05:00:00Z,71935339,104.091749
1,102.71,105.850,102.41,321365,105.75,2016-01-05T05:00:00Z,58690536,103.400868
2,100.70,102.370,99.87,409164,100.56,2016-01-06T05:00:00Z,71079827,100.843959
3,96.45,100.130,96.43,462836,98.68,2016-01-07T05:00:00Z,85996453,98.011371
4,96.96,99.110,96.76,419555,98.55,2016-01-08T05:00:00Z,75066292,97.897369


In [2]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data['close'].values.reshape(-1,1))

In [3]:
import numpy as np

def create_dataset(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [4]:
time_step = 100
X, y = create_dataset(scaled_data, time_step)

train_size=0.8
X_train, X_test = X[:int(X.shape[0]*train_size)], X[int(X.shape[0]*train_size):]
y_train, y_test = y[:int(y.shape[0]*train_size)], y[int(y.shape[0]*train_size):]

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(units=64))
model.add(Dense(units=64))
model.add(Dense(units=1))

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=64)

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
original_data = stock_data['Close'].values
predicted_data = np.empty_like(original_data)
predicted_data[:] = np.nan
predicted_data[-len(predictions):] = predictions.reshape(-1)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(original_data, label='Original Data')
plt.plot(predicted_data, label='Predicted Data')
plt.legend()
plt.show()

In [ ]:
new_predictions = model.predict(X_test[-90:])
new_predictions = scaler.inverse_transform(new_predictions)

predicted_data = np.append(predicted_data, new_predictions)
predicted_data[-90:]

In [ ]:
plt.plot(original_data, label='Original Data')
plt.plot(predicted_data, label='Predicted Data')
plt.legend()
plt.show()